In [1]:

# import sklearn
# import sklearn.ensemble
# import sklearn.metrics
# import sklearn.feature_extraction
# from sklearn.pipeline import make_pipeline
import pointing_game_util

In [2]:
import pandas as pd


from gpt2outputdataset.detector_radford import DetectorRadford
# from detectgpt.detector_detectgpt import DetectorDetectGPT
train = pd.read_pickle("./dataset_train.pkl")
documents = train["answer"]
gold_labels = train["author"] == "chatgpt_answers"

In [3]:
detector = DetectorRadford()

In [4]:

# c = make_pipeline(detector)

In [5]:
# c = make_pipeline(detector)
# from lime.lime_text import LimeTextExplainer
# explainer = LimeTextExplainer(class_names=("machine", "human"), bow=False, split_expression= r"\s",mask_string=detector.get_pad_token()) # TODO
# # exp = explainer.explain_instance(text_gpt2, c.predict_proba, num_features=10, num_samples=1000)

In [6]:
# exp = explainer.explain_instance(text_gpt2, c.predict_proba, num_features=10, num_samples=1000)

In [7]:
# exp.show_in_notebook()

In [8]:
documents.to_list()[0:10]

["As a former substance abuse counselor I will say most of these answers are spot on and do n't need to be said again However , I wonder if your question is n't really ' why do people like the taste of alcohol ? ' If that 's the case , then the answer is simple enough . Alcohol is an acquired taste ( much like coffee is ) . We have to force ourselves to like it . Even so , some people ( even alcoholics ) go their while lives drinking and can never stand the taste of it . They do so because of the way it makes them feel . It is mostly because of this people create mixed drinks ( e.g. long island iced tea , etc ) , to keep the alcohol flavor from being strong while still giving the desired effect .",
 "I can walk up to a cop and tell him that I take ecstasy every day , and he has to prove that I do . The only thing he can do is search me and possibly get a warrant if he can convince a judge that I actually told him that and I was n't just being obtuse . It 's not illegal to be on , or to

In [9]:
from fi_explainer import FI_Explainer
from sklearn.pipeline import make_pipeline
from lime.lime_text import LimeTextExplainer
import re

class LIME_Explainer(FI_Explainer):
    def __init__(self, num_features=10, num_samples=1000, detector_class=DetectorRadford):
        self.num_features = num_features
        self.num_samples = num_samples
        self.detector = detector_class()
     #   c = make_pipeline(detector)
        self.explainer = LimeTextExplainer(class_names=("machine", "human"), bow=False, split_expression= r"\s",mask_string=self.detector.get_pad_token()) # TODO
    def get_explanation(self, document):
        return self.explainer.explain_instance(document, self.detector.predict_proba, num_features=self.num_features, num_samples=self.num_samples, top_labels=2)
    def get_fi_scores(self, document):
        return self.get_explanation_cached(document).as_map()
    def get_fi_scores_batch(self, documents):
        return [self.get_fi_scores(document) for document in documents]

In [10]:
explainer = LIME_Explainer()

splitter = re.compile(r'(%s)|$' % explainer.explainer.split_expression)
tokenizer_lime = lambda document : [s for s in splitter.split(document) if s and s!= " "]

In [11]:
hybrid_documents, tokenized_hybrid_documents, GT = pointing_game_util.hybrid(documents.to_list(), gold_labels.to_list(), word_tokenizer=tokenizer_lime)
predictions_hybrid = detector.predict_proba(hybrid_documents).argmax(axis=1)

In [12]:
# explanations = explainer.get_explanations_cached(hybrid_documents) just for debug

In [13]:
pointing_game_acc = pointing_game_util.get_pointing_game_acc(hybrid_documents, explainer, predictions_hybrid, GT)

In [14]:
pointing_game_acc

0.45454545454545453

In [15]:
# for idx_top_word, tokenized, GT_doc, explanation, prediction_hybrid in zip(indices_top_word,tokenized_hybrid_documents,GT, explanations, predictions_hybrid):
#     print(idx_top_word, tokenized[idx_top_word], GT_doc[idx_top_word])
#     print("hit" if GT_doc[idx_top_word] == prediction_hybrid else "miss")
#     explanation.show_in_notebook()